Building likelihood maps for a set of representative images.

> TODO: use images from dataset and superimpose the bounding boxes on them

In [ ]:
import retinoto_py as fovea
args = fovea.Params(do_fovea=True, batch_size=1, subset_factor=25)
args

In [ ]:
dataset = 'bbox'
model_filename = args.data_cache /  f'32_fovea_model_name={args.model_name}_dataset={dataset}.pth'
model = fovea.load_model(args, model_filename=model_filename)

In [ ]:

VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
val_dataset = fovea.get_dataset(args, VAL_DATA_DIR)
# val_loader = fovea.get_loader(args, val_dataset)



In [ ]:
parquet_filename = args.data_cache / '42_likelihood_maps.parquet'
resolution = (50, 30)
resolution = (20, 20)
size_ratio = 1.

if parquet_filename.exists():
    results = fovea.pd.read_json(parquet_filename)
else:

    n_dataset = len(val_dataset)

    with fovea.torch.no_grad():
        all_results = []
        for i_batch, (image, true_label) in fovea.tqdm(enumerate(val_dataset), total=n_dataset):
            pos_H, pos_W, outputs = fovea.compute_likelihood_map(args, model, image, size_ratio=size_ratio, resolution=resolution)
            logit_label = outputs[:, true_label]
            logit_label = logit_label.cpu()            
            all_results.append({'pos_H':pos_H, 'pos_W':pos_W, 'outputs':outputs})
        results = fovea.pd.DataFrame(all_results)
        results.to_parquet(parquet_filename)